In [33]:
# libraries
#!pip install rpy2
#!pip install pandas
#!pip install keras
#!pip install imblearn

    100% |████████████████████████████████| 92kB 660kB/s 
  Running setup.py bdist_wheel for imbalanced-learn ... - \ | / done
  Stored in directory: /Users/Kozodoi/Library/Caches/pip/wheels/b8/20/bd/0b775f7e5d413ac72562b1a5126598bcb6e0eae10da659be9f
Successfully built imbalanced-learn


In [1]:
# libraries
import rpy2.robjects as robjects
import pandas as pd
import keras
from keras.layers import Input, Dense, Embedding, concatenate, Flatten, Dropout, Reshape
from keras.layers.normalization import BatchNormalization
from keras.models import Model
from keras.regularizers import l2
from keras.optimizers import Adam

Using TensorFlow backend.


## 1. DATA PREPARATION

In [2]:
# load the data
path = "/Users/Kozodoi/Documents/Competitions/DSG_2017/"
known   = pd.read_csv(path + "data/train.csv")
unknown = pd.read_csv(path + "data/test.csv")

In [3]:
# keep only Flow songs in the data
# this proves to predict better, but some information is lost
known = known.query("listen_type == 1")
known.shape

(2319611, 15)

In [4]:
# limiting the maximal number of songs to 100 - works bad
#known = known.groupby(["user_id"]).tail(100)
#known.shape

In [5]:
# Prepare the data
# Create a placeholder for the IDs new in the test data
newUsers   = list(unknown.user_id[~unknown.user_id.isin(known.user_id)])       + list(known.user_id.value_counts().keys()[known.user_id.value_counts()     == 1])
newSongs   = list(unknown.media_id[~unknown.media_id.isin(known.media_id)])    + list(known.media_id.value_counts().keys()[known.media_id.value_counts()   == 1])[-1000:]
newArtists = list(unknown.artist_id[~unknown.artist_id.isin(known.artist_id)]) + list(known.artist_id.value_counts().keys()[known.artist_id.value_counts() == 1])[-1000:]
newGenres  = list(unknown.genre_id[~unknown.genre_id.isin(known.genre_id)])    + list(known.genre_id.value_counts().keys()[known.genre_id.value_counts()   == 1])[-1000:]
newAlbums  = list(unknown.album_id[~unknown.album_id.isin(known.album_id)])    + list(known.album_id.value_counts().keys()[known.album_id.value_counts() == 1])[-1000:]
newContext = list(unknown.context_type[~unknown.context_type.isin(known.context_type)]) + list(known.context_type.value_counts().keys()[known.context_type.value_counts() == 1])[-1000:]

# In particular, assign IDs from 0 to N for users and songs
# Use enumerate() to create a list of the new and original IDs
users   = list(enumerate([i for i in known.user_id.unique()   if i not in newUsers]))
songs   = list(enumerate([i for i in known.media_id.unique()  if i not in newSongs]))
artists = list(enumerate([i for i in known.artist_id.unique() if i not in newArtists]))
genres  = list(enumerate([i for i in known.genre_id.unique()  if i not in newGenres]))
albums  = list(enumerate([i for i in known.album_id.unique()  if i not in newAlbums]))
context = list(enumerate([i for i in known.context_type.unique() if i not in newContext]))

# Create a dictionary with old IDs to new IDs
userid2idx    = {o:i for i,o in users}
songid2idx    = {o:i for i,o in songs}
artistid2idx  = {o:i for i,o in artists}
genreid2idx   = {o:i for i,o in genres}
albumid2idx   = {o:i for i,o in albums}
contextid2idx = {o:i for i,o in context}

# Update with new/rare entries
userid2idx.update({o:(max(userid2idx.values())+1)       for o in newUsers})
songid2idx.update({o:(max(songid2idx.values())+1)       for o in newSongs})
artistid2idx.update({o:(max(artistid2idx.values())+1)   for o in newArtists})
genreid2idx.update({o:(max(genreid2idx.values())+1)     for o in newGenres})
albumid2idx.update({o:(max(albumid2idx.values())+1)     for o in newAlbums})
contextid2idx.update({o:(max(contextid2idx.values())+1) for o in newContext})

# Create id variable with the new IDs (known)
known['userIdx']    = known.user_id.apply(lambda x:       userid2idx[x])
known['songIdx']    = known.media_id.apply(lambda x:      songid2idx[x])
known['artistIdx']  = known.artist_id.apply(lambda x:     artistid2idx[x])
known['genreIdx']   = known.genre_id.apply(lambda x:      genreid2idx[x])
known['albumIdx']   = known.album_id.apply(lambda x:      albumid2idx[x])
known['contextIdx'] = known.context_type.apply(lambda x:  contextid2idx[x])

# Create id variable with the new IDs (unknown)
unknown['userIdx']    = unknown.user_id.apply(lambda x:      userid2idx[x])
unknown['songIdx']    = unknown.media_id.apply(lambda x:     songid2idx[x])
unknown['artistIdx']  = unknown.artist_id.apply(lambda x:    artistid2idx[x])
unknown['genreIdx']   = unknown.genre_id.apply(lambda x:     genreid2idx[x])
unknown['albumIdx']   = unknown.album_id.apply(lambda x:     albumid2idx[x])
unknown['contextIdx'] = unknown.context_type.apply(lambda x: contextid2idx[x])

In [6]:
# partition train/test data: last 3 songs per user go to validation (stage 1)
# model predicts better if trained on full known sample without validation (stage 2)
ts = known.groupby(["userIdx"]).tail(3)
tr = known.groupby(["userIdx"], group_keys=False).apply(lambda x: x[:-3])

# move songs that appear only in ts to tr
strayObs = ts.songIdx.isin(tr.songIdx) & ts.userIdx.isin(tr.userIdx) & ts.artistIdx.isin(tr.artistIdx) & ts.genreIdx.isin(tr.genreIdx) & ts.albumIdx.isin(tr.albumIdx) & ts.contextIdx.isin(tr.contextIdx)
tr = tr.append(ts[~strayObs])
ts = ts[strayObs]

In [8]:
# saving the data samples
tr.to_csv(path + "data/tr.csv", index = False)
ts.to_csv(path + "data/ts.csv", index = False)

## 2. MODELING

### 2.1. INITIALIZING

In [7]:
# create an input layer with one row of IDs
user_in    = Input(shape = (1,), dtype = 'int64',   name = "user_in")
song_in    = Input(shape = (1,), dtype = 'int64',   name = "song_in")
artist_in  = Input(shape = (1,), dtype = 'int64',   name = "artist_in")
genre_in   = Input(shape = (1,), dtype = 'int64',   name = "genre_in")
album_in   = Input(shape = (1,), dtype = 'int64',   name = "album_in")
context_in = Input(shape = (1,), dtype = 'int64',   name = "context_in")
num_in     = Input(shape = (1,), dtype = 'float32', name = "num_in")

# Reshaping numeric features
n = Reshape((1,1))(num_in)

# Create an embedding assigning k latent factors to each ID
# These will be optimized
# A regulariztaion is added to avoid very large weights
n_users   = tr.userIdx.nunique()
n_songs   = tr.songIdx.nunique()
n_artists = tr.artistIdx.nunique()
n_genres  = tr.genreIdx.nunique()
n_albums  = tr.albumIdx.nunique()
n_context = tr.contextIdx.nunique()

# Embeddings creation
u = Embedding(n_users,   50, input_length=1, embeddings_regularizer=l2(1e-5))(user_in)
s = Embedding(n_songs,   50, input_length=1, embeddings_regularizer=l2(1e-5))(song_in)
a = Embedding(n_artists, 50, input_length=1, embeddings_regularizer=l2(1e-5))(artist_in)
g = Embedding(n_genres,  50, input_length=1, embeddings_regularizer=l2(1e-5))(genre_in)
l = Embedding(n_albums,  50, input_length=1, embeddings_regularizer=l2(1e-5))(album_in)
c = Embedding(n_context, 50, input_length=1, embeddings_regularizer=l2(1e-5))(context_in)

# Specify what to do with the layers
#x = concatenate([u, s, a, g, l, c, n]) # with one numeric feature
x = concatenate([u, s])
x = Flatten()(x)
x = Dense(64, activation='relu')(x)
x = Dropout(0.5)(x)
x = BatchNormalization()(x)
x = Dense(32, activation='relu')(x) 
x = Dropout(0.5)(x)
x = Dense(1, activation = "sigmoid")(x)

# Then we specify the model that we want to use
#model = Model([user_in, song_in, artist_in, genre_in, album_in, context_in, num_in], x) # with one numeric feature
model = Model([user_in, song_in], x) 
#model.compile(Adam(0.001), loss = "binary_crossentropy", metrics = ['accuracy'])
model.compile(optimizer="Adagrad", loss="binary_crossentropy", metrics = ['accuracy'])

### 2.2. FIRST STAGE

In [8]:
# run the estimations on training data
model.fit([tr.userIdx, tr.songIdx], tr.is_listened, 
validation_data = ([ts.userIdx, ts.songIdx], ts.is_listened),
batch_size = int(len(tr)/100), epochs = 10)

Train on 2281342 samples, validate on 38269 samples
Epoch 1/10
2281342/2281342 [==============================] - 101s - loss: 0.5412 - acc: 0.7448 - val_loss: 0.6988 - val_acc: 0.6249
Epoch 2/10
2281342/2281342 [==============================] - 124s - loss: 0.4987 - acc: 0.7702 - val_loss: 0.6860 - val_acc: 0.6960
Epoch 3/10
2281342/2281342 [==============================] - 124s - loss: 0.4864 - acc: 0.7771 - val_loss: 0.6551 - val_acc: 0.7337
Epoch 4/10
2281342/2281342 [==============================] - 124s - loss: 0.4771 - acc: 0.7822 - val_loss: 0.6225 - val_acc: 0.7013
Epoch 5/10
2281342/2281342 [==============================] - 125s - loss: 0.4692 - acc: 0.7864 - val_loss: 0.5798 - val_acc: 0.7333
Epoch 6/10
2281342/2281342 [==============================] - 127s - loss: 0.4621 - acc: 0.7905 - val_loss: 0.5545 - val_acc: 0.7331
Epoch 7/10
2281342/2281342 [==============================] - 127s - loss: 0.4553 - acc: 0.7939 - val_loss: 0.5606 - val_acc: 0.7284
Epoch 8/10
228134

In [9]:
# predict validation data
pred = pd.DataFrame()
pred["user_id"]  = ts.user_id
pred["media_id"] = ts.media_id
pred["is_listened"] = model.predict([ts.userIdx, ts.songIdx])
pred.to_csv(path + "pred_valid/deep_keras_06.csv", index = False)
pred.head(5)

,user_id,media_id,is_listened
41876,1927,127539469,0.684048
41877,1927,127539479,0.783574
85967,6272,116866548,0.803330
85968,6272,116866550,0.417653
126942,8177,130105294,0.980958


### 2.3. SECOND STAGE

In [8]:
# run the estimations on full known data
model.fit([known.userIdx, known.songIdx], known.is_listened,
batch_size = int(known.shape[0]/100), epochs = 10)

Epoch 1/10
2319611/2319611 [==============================] - 61s - loss: 0.5421 - acc: 0.7359    
Epoch 2/10
2319611/2319611 [==============================] - 61s - loss: 0.4980 - acc: 0.7669    
Epoch 3/10
2319611/2319611 [==============================] - 60s - loss: 0.4865 - acc: 0.7746    
Epoch 4/10
2319611/2319611 [==============================] - 59s - loss: 0.4788 - acc: 0.7795    
Epoch 5/10
2319611/2319611 [==============================] - 62s - loss: 0.4725 - acc: 0.7830    
Epoch 6/10
2319611/2319611 [==============================] - 61s - loss: 0.4670 - acc: 0.7859    
Epoch 7/10
2319611/2319611 [==============================] - 61s - loss: 0.4629 - acc: 0.7879    
Epoch 8/10
2319611/2319611 [==============================] - 39s - loss: 0.4589 - acc: 0.7896    
Epoch 9/10
2319611/2319611 [==============================] - 38s - loss: 0.4557 - acc: 0.7912    
Epoch 10/10
2319611/2319611 [==============================] - 39s - loss: 0.4524 - acc: 0.7931    


In [9]:
# predict unknown data
pred = pd.DataFrame()
pred["sample_id"] = unknown.sample_id
pred["is_listened"] = model.predict([unknown.userIdx, unknown.songIdx])
pred.to_csv(path + "pred_unknown/deep_keras_06.csv", index = False)
pred.head(5)

,sample_id,is_listened
0,0,0.986938
1,1,0.752610
2,2,0.759758
3,3,0.536454
4,4,0.979069
